# Backpropagation

In [ ]:
import numpy as np
import pandas as pd
import time

Pada post-test kali ini akan membandingkan dua jenis fungsi aktivasi yang biasa digunakan dalam backpropogation

In [ ]:
#Fungsi Aktivasi Sigmoid dengan turunannya
def sig(X):
    X = np.clip(X, -500, 500)  # Prevent overflow in exponential calculation
    return [1 / (1 + np.exp(-x)) for x in X]

def sigd(X):
  output = []
  for i, x in enumerate(X):
    s = sig([x])[0]
    output.append(s * (1 - s))
  return output

#Fungsi Aktivasi Hyperbolic Tangent dengan turunannya
def tanh(X):
    return [np.tanh(x) for x in X]

def tanhd(X):
    output = []
    for x in X:
        t = np.tanh(x)
        output.append(1 - t ** 2)
    return output

In [ ]:
def onehot_enc(lbl, min_val=0):
  mi = min(lbl)
  enc = np.full((len(lbl), max(lbl) - mi + 1), min_val, np.int8)

  for i, x in enumerate(lbl):
    enc[i, x - mi] = 1

  return enc

def onehot_dec(enc, mi=0):
  return [np.argmax(e) + mi for e in enc]

### a) Fungsi *Training* Backpropagation

Tulis kode ke dalam *cell* di bawah ini:

In [13]:
import time
def bp_fit_sig(X, target, layer_conf, max_epoch, max_error=.1, learn_rate=.1, print_per_epoch=100):
  start_time = time.time()
  np.random.seed(1)
  # Lengkapi kode Dibawah ini
  nin = [np.empty(i) for i in layer_conf]
  n = [np.empty(j + 1) if i < len(layer_conf) - 1 else np.empty(j) for i, j in enumerate(layer_conf)]

  w = [np.random.rand(layer_conf[i] + 1, layer_conf[i + 1]) for i in range(len(layer_conf) - 1)]

  dw = [np.empty((layer_conf[i] + 1, layer_conf[i + 1])) for i in range(len(layer_conf) - 1)]
  d = [np.empty(s) for s in layer_conf[1:]]
  din = [np.empty(s) for s in layer_conf[1:-1]]
  epoch = 0
  mse = 1
  for i in range(0, len(n)-1):
    n[i][-1] = 1
  while (max_epoch == -1 or epoch < max_epoch) and mse > max_error:
    epoch += 1
    mse = 0
    for r in range(len(X)):
      n[0][:-1] = X[r]
      for L in range(1, len(layer_conf)):
        nin[L] = np.dot(n[L-1], w[L-1])
        n[L][:len(nin[L])] = sig(nin[L])
      e = target[r] - n[-1]
      mse += sum(e ** 2)
      d[-1] = e * sigd(nin[-1])
      dw[-1] = learn_rate * d[-1] * n[-2].reshape((-1, 1))
      for L in range(len(layer_conf) - 1, 1, -1):

        # Lengkapi kode Dibawah ini
        din[L-2] = np.dot(d[L-1],np.transpose(w[L-1][:-1]))
        d[L-2] = din[L-2] * np.array(sigd(nin[L-1]))
        dw[L-2] = (learn_rate * d[L-2]) *n[L-2].reshape((-1, 1))

      w += dw
    mse /= len(X)
    if print_per_epoch > -1 and epoch % print_per_epoch == 0:
      print(f'Epoch {epoch}, MSE: {mse}')
  execution = time.time() - start_time
  print("Waktu eksekusi: %s detik" % execution)
  return w, epoch, mse

In [ ]:
#Membuat fungsi training backpropagation dengan menggunakan fungsi aktivasi tanh
import time
def bp_fit_tanh(X, target, layer_conf, max_epoch, max_error=.1, learn_rate=.1, print_per_epoch=100):
  start_time = time.time()
  np.random.seed(1)
  # Lengkapi kode Dibawah ini
  nin = [np.empty(i) for i in layer_conf]
  n = [np.empty(j + 1) if i < len(layer_conf) - 1 else np.empty(j) for i, j in enumerate(layer_conf)]

  w = [np.random.rand(layer_conf[i] + 1, layer_conf[i + 1]) for i in range(len(layer_conf) - 1)]
  dw = [np.empty((layer_conf[i] + 1, layer_conf[i + 1])) for i in range(len(layer_conf) - 1)]
  d = [np.empty(s) for s in layer_conf[1:]]
  din = [np.empty(s) for s in layer_conf[1:-1]]
  epoch = 0
  mse = 1
  for i in range(0, len(n)-1):
    n[i][-1] = 1
  while (max_epoch == -1 or epoch < max_epoch) and mse > max_error:
    epoch += 1
    mse = 0
    for r in range(len(X)):
      n[0][:-1] = X[r]
      for L in range(1, len(layer_conf)):
        nin[L] = np.dot(n[L-1], w[L-1])
        n[L][:len(nin[L])] = tanh(nin[L])
      e = target[r] - n[-1]
      mse += sum(e ** 2)
      d[-1] = e * tanhd(nin[-1])
      dw[-1] = learn_rate * d[-1] * n[-2].reshape((-1, 1))
      for L in range(len(layer_conf) - 1, 1, -1):

        # Lengkapi kode Dibawah ini
        din[L-2] = np.dot(d[L-1],np.transpose(w[L-1][:-1]))
        d[L-2] = din[L-2] * np.array(tanhd(nin[L-1]))
        dw[L-2] = (learn_rate * d[L-2]) *n[L-2].reshape((-1, 1))

      w += dw
    mse /= len(X)
    if print_per_epoch > -1 and epoch % print_per_epoch == 0:
      print(f'Epoch {epoch}, MSE: {mse}')
  execution = time.time() - start_time
  print("Waktu eksekusi: %s detik" % execution)
  return w, epoch, mse

### b) Fungsi *Testing* Backpropagation

Tulis kode ke dalam *cell* di bawah ini:

In [14]:
def bp_predict_sig(X, w):
  n = [np.empty(len(i)) for i in w]
  nin = [np.empty(len(i[0])) for i in w]
  predict = []
  n.append(np.empty(len(w[-1][0])))
  for x in X:
    n[0][:-1] = x
    for L in range(0, len(w)):
      nin[L] = np.dot(n[L], w[L])
      n[L + 1][:len(nin[L])] = sig(nin[L])
    predict.append(n[-1].copy())
  return predict

In [15]:
#Membuat fungsi testing backpropagation dengan menggunakan fungsi aktivasi tanh
def bp_predict_tanh(X, w):
    n = [np.empty(len(i)) for i in w]
    nin = [np.empty(len(i[0])) for i in w]
    predict = []
    n.append(np.empty(len(w[-1][0])))

    for x in X:
        n[0][:-1] = x
        for L in range(0, len(w)):
            nin[L] = np.dot(n[L], w[L])
            n[L + 1][:len(nin[L])] = tanh(nin[L])
        predict.append(n[-1].copy())

    return predict

### c) Klasifikasi dataset wine


Lakukan pelatihan pada dataset wine dengan menggunakan 2 fungsi pelatihan yang telah dibuat!

Konfigurasi kedua pelatihan harus sama (epoch, hidden layer, learning rate, dll).
Akurasi yang diharapkan di setiap pelatihan adalah > 0.98

In [20]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score

wine = datasets.load_wine()
X = minmax_scale(wine.data)
Y = onehot_enc(wine.target)

X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=.3,random_state=1)
#Isi jumlah layer yang digunakan dengan jumlah hidden layer #
w, ep, mse = bp_fit_sig(X_train, y_train, layer_conf=(13, 10, 3),learn_rate=0.1, max_epoch=10, max_error=0.1, print_per_epoch=25)

print(f'Epochs: {ep}, MSE: {mse}')

predict = bp_predict_sig(X_test, w)
predict = onehot_dec(predict)
y_test = onehot_dec(y_test)
accuracy = accuracy_score(predict, y_test)

print('Output:', predict)
print('True :', y_test)
print('Accuracy:', accuracy)

Waktu eksekusi: 0.30729198455810547 detik
Epochs: 10, MSE: 1.954027674036451
Output: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
True : [2, 1, 0, 1, 0, 2, 1, 0, 2, 1, 0, 0, 1, 0, 1, 1, 2, 0, 1, 0, 0, 1, 2, 1, 0, 2, 0, 0, 0, 2, 1, 2, 2, 0, 1, 1, 1, 1, 1, 0, 0, 1, 2, 0, 0, 0, 1, 0, 0, 0, 1, 2, 2, 0]
Accuracy: 0.42592592592592593


In [17]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score

wine = datasets.load_wine()
X = minmax_scale(wine.data)
Y = onehot_enc(wine.target)

X_train, X_test, y_train, y_test = train_test_split(X, Y,
test_size=.3,random_state=1)
#Isi jumlah layer yang digunakan dengan jumlah hidden layer #
w, ep, mse = bp_fit_tanh(X_train, y_train, layer_conf=(13, 3, 3),learn_rate=0.1, max_epoch=10, max_error=0.01, print_per_epoch=25)

print(f'Epochs: {ep}, MSE: {mse}')

predict = bp_predict_tanh(X_test, w)
predict = onehot_dec(predict)
y_test = onehot_dec(y_test)
accuracy = accuracy_score(predict, y_test)

print('Output:', predict)
print('True :', y_test)
print('Accuracy:', accuracy)

Waktu eksekusi: 0.3185310363769531 detik
Epochs: 10, MSE: 1.4600188247380315
Output: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
True : [2, 1, 0, 1, 0, 2, 1, 0, 2, 1, 0, 0, 1, 0, 1, 1, 2, 0, 1, 0, 0, 1, 2, 1, 0, 2, 0, 0, 0, 2, 1, 2, 2, 0, 1, 1, 1, 1, 1, 0, 0, 1, 2, 0, 0, 0, 1, 0, 0, 0, 1, 2, 2, 0]
Accuracy: 0.42592592592592593


# Pertanyaan

1.  Apa perbedaan dari penggunaan fungsi aktivasi sigmoid dengan fungsi aktivasi hyperbolic tangent?
2. Coba jelaskan alasan dari perbedaan tersebut sebisa kalian

# Jawaban

1.  Fungsi aktivasi sigmoid mengubah setiap nilai input menjadi angka antara 0 dan 1, mirip seperti grafik berbentuk "S". Ketika nilai input semakin besar, hasilnya mendekati 1, dan ketika input semakin kecil, hasilnya mendekati 0. Sigmoid sering dipakai karena cocok untuk memodelkan probabilitas atau hasil biner. Namun, kelemahan utamanya adalah terjadinya "vanishing gradient" atau peluruhan nilai turunan pada nilai input yang terlalu besar atau terlalu kecil, membuat proses pembelajaran jadi lambat pada lapisan-lapisan yang lebih dalam dalam jaringan saraf. Ini bisa menyebabkan model butuh waktu lama untuk memahami pola, terutama dalam jaringan saraf yang dalam.



2. Fungsi aktivasi tanh mirip dengan sigmoid dalam hal bentuk, tapi memberikan hasil antara -1 dan 1. Ini berarti tanh memiliki nilai negatif untuk input negatif dan nilai positif untuk input positif, yang bisa membantu jaringan saraf belajar lebih cepat. Karena rentangnya yang lebih luas dibandingkan sigmoid, fungsi tanh mengurangi masalah vanishing gradient lebih baik, karena outputnya lebih "sentral" di sekitar 0. Jadi, informasi bisa mengalir lebih mudah ke lapisan berikutnya, memungkinkan pembelajaran lebih cepat, khususnya dalam jaringan saraf yang dalam.  